# Libraries

In [ ]:
!pwd

## Import

In [1]:
# Libraries

import sys
sys.path.append('/home/ec2-user/SageMaker/Langfuse/multi-agent-awsStrands-main/taubench/data/ma-bench/')
sys.path.append('/home/ec2-user/SageMaker/Langfuse/multi-agent-awsStrands-main/taubench/data/tau-bench/')

import os
import json
import importlib
import argparse
import warnings
import re
import base64
import uuid

import boto3
from botocore.config import Config

# Strands imports
from strands import Agent, tool
from strands.models import BedrockModel
from strands.multiagent import GraphBuilder
from strands.telemetry.config import StrandsTelemetry

# Parameters

In [2]:
# setup boto3 config to allow for retrying
region_name = "us-west-2"
my_config = Config(
    region_name = region_name,
    signature_version = 'v4',
    retries = {
        'max_attempts': 50,
        'mode': 'standard'
    }
)

# select domain
domain = "airline"
# # Parse command line arguments
# parser = argparse.ArgumentParser(description='Run agent with specified domain')
# parser.add_argument('--domain', type=str, default=domain, 
#                     help='Domain to use (e.g., "airline", "retail")')
# args = parser.parse_args()

# # Update domain if provided via command line
# domain = args.domain

######################### LANGFUSE SETUP ########################
# Langfuse credentials
os.environ["LANGFUSE_PUBLIC_KEY"] = "[ADD PUBLIC KEY HERE]"
os.environ["LANGFUSE_SECRET_KEY"] = "[ADD SECRET KEY HERE]"
os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com"



# Build Basic Auth header
LANGFUSE_AUTH = base64.b64encode(
    f"{os.environ.get('LANGFUSE_PUBLIC_KEY')}:{os.environ.get('LANGFUSE_SECRET_KEY')}".encode()
).decode()

# Configure OpenTelemetry endpoint & headers
os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = os.environ.get("LANGFUSE_HOST") + "/api/public/otel/"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"

# Initialize OpenTelemetry BEFORE creating Strands agent

strands_telemetry = StrandsTelemetry()
strands_telemetry.setup_otlp_exporter()
# strands_telemetry.setup_console_exporter()  # Print traces to console
######################### LANGFUSE SETUP ########################

# Utils

In [3]:
def import_domain_tools(domain):
    """
    Dynamically import tools based on the domain
    """
    tools_module = importlib.import_module(f'mabench.environments.{domain}.tools_strands')
    tools_dict = {}
    
    # Get all attributes from the tools module
    for attr_name in dir(tools_module):
        if attr_name.startswith('__'):
            continue
        
        try:
            # Try to import each tool
            tool_module = importlib.import_module(f'mabench.environments.{domain}.tools_strands.{attr_name}')
            # Get the tool function from the module
            if hasattr(tool_module, attr_name):
                tools_dict[attr_name] = getattr(tool_module, attr_name)
        except (ImportError, AttributeError):
            pass
    
    return tools_dict


def run_user_agent(user, agent):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")

        user_response_text = "Hi"

        while "###STOP###" not in user_response_text:
            print("\n\n******** Agent ********\n")
            agent_response = agent(user_response_text)
            print(agent.trace_attributes)
            agent_response_thinking, agent_response_text = extract_thinking_and_response(str(agent_response))
            print("\n\n******** User *********\n")
            user_response = user(agent_response_text)
            user_response_thinking, user_response_text = extract_thinking_and_response(str(user_response))
        return agent.messages
    
def extract_thinking_and_response(text):
    match = re.search(r'<thinking>(.*?)</thinking>(.*)', text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip(), match.group(2).strip()
    else:
        return "", text.strip()

In [4]:
# Import domain-specific modules
# try:
# Import wiki
wiki_module = importlib.import_module(f'tau_bench.envs.{domain}.wiki')
WIKI = getattr(wiki_module, 'WIKI')

# Import data and tasks
importlib.import_module(f'tau_bench.envs.{domain}.data')
importlib.import_module(f'tau_bench.envs.{domain}.tasks')

# Import tools
domain_tools = import_domain_tools(domain)

print(f"Successfully loaded modules for domain: {domain}")
# except ImportError as e:
#     print(f"Error: Could not import modules for domain '{domain}'. Error: {e}")
#     print("Available domains may include: airline, retail")
#     sys.exit(1)

Successfully loaded modules for domain: airline


# User

In [5]:
def user_prompt(instruction):
    
    system_prompt_template = """
You are a user interacting with an agent.

{instruction}

Rules:
- generate a one line User Response to simulate the user's message (this message will be sent to the agent).
- Do not give away all the instruction at once. Only provide the information that is necessary for the current step.
- Do not hallucinate information that is not provided in the instruction. For example, if the agent asks for the order id but it is not mentioned in the instruction, do not make up an order id, just say you do not remember or have it.
- If the instruction goal is satisified, generate '###STOP###' as a standalone message without anything else to end the conversation.
- Do not repeat the exact instruction in the conversation. Instead, use your own words to convey the same information.
- Try to make the conversation as natural as possible, and stick to the personalities in the instruction.
"""

    prompt = system_prompt_template.format(instruction = instruction)

    return prompt

def user_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )

def simulated_user_tracing(user_id, session_id, domain):

    trace_attributes = {
        "user.id": user_id, 
        "session.id": session_id,
        "langfuse.tags": [
            user_id,
            session_id,
            f"awsStrands-singleAgent_multiTurn-{domain}",
        ]
    }

    return trace_attributes

def simulated_user(instruction, user_id, session_id, domain):

    prompt = user_prompt(instruction)
    model = user_model()
    trace_attributes = simulated_user_tracing(user_id, session_id, domain)

    return Agent(
        name = f"awsStrands-singleAgent_multiTurn_simulatedUser-{domain}-{user_id}-{session_id}",
        model = model,
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )

# Agent

## user agent

In [6]:
user_tool_list = [
    'get_user_details', 
    'send_certificate', 
    'think'
]
user_agent_tools = [domain_tools[key] for key in user_tool_list]

def user_agent_prompt():
    
    system_prompt_template = """
You are the User Agent for a travel website, specializing in customer data management and user profile operations.
Your primary responsibilities include retrieving user information and managing customer benefits.
Use the provided tools to assist queries for user information.

<capabilities>
- You can access user profiles and retrieve customer details using the get_user_details tool
- You can issue certificates and benefits to users through the send_certificate tool
- You can use the think tool for internal reasoning
</capabilities>

<instructions>
- You should not use made-up or placeholder arguments.
</instructions>

<policy>
{policy}
</policy>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt

def user_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def user_agent_tracing(domain):

    trace_attributes = {
        "langfuse.tags": [
            f"awsStrands-multiAgent_singleTurn_userAgent-{domain}",
        ]
    }

    return trace_attributes


def user_react_agent(tools):

    prompt = user_agent_prompt()
    model = user_agent_model()
    trace_attributes = user_agent_tracing(domain)

    return Agent(
        name = f"awsStrands-multiAgent_singleTurn_userAgent-{domain}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )


@tool
def user_information_manager(query: str) -> str:
    """
    Process and respond to queries about user profiles, customer details, and sending certificates.
    Use for ANY user-related queries including account information and sending certificates.
    
    Args:
        query: A question requiring access to user profiles, account details, or sending certificates
        
    Returns:
        Detailed user information or confirmation of benefit actions
    """
    try:
        # Create the specialized User Agent using the configuration you've provided
        user_agent = user_react_agent(user_agent_tools)
        
        # Call the agent and return its response
        response = user_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in user_information_manager: {str(e)}"

## flight agent

In [7]:
flight_tool_list = [
    'search_direct_flight', 
    'search_onestop_flight', 
    'list_all_airports', 
    'think'
]
flight_agent_tools = [domain_tools[key] for key in flight_tool_list]

def flight_agent_prompt():
    
    system_prompt_template = """
You are the Flight Agent for a travel website, specializing in flight search operations and airport information management.
Your expertise lies in finding flight routes and providing accurate airport data to support the reservation process.
Use the provided tools to search for flights.

<capabilities>
- You can search for direct flights between airports using the search_direct_flight tool
- You can find connecting flights with one stop using the search_onestop_flight tool
- You can provide comprehensive airport information via the list_all_airports tool
- You can use the think tool for reasoning
</capabilities>

<instructions>
- You should not use made-up or placeholder arguments.
</instructions>

<policy>
{policy}
</policy>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt

def flight_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def flight_agent_tracing(domain):

    trace_attributes = {
        "langfuse.tags": [
            f"awsStrands-multiAgent_singleTurn_flightAgent-{domain}",
        ]
    }

    return trace_attributes


def flight_react_agent(tools):

    prompt = flight_agent_prompt()
    model = flight_agent_model()
    trace_attributes = flight_agent_tracing(domain)

    return Agent(
        name = f"awsStrands-multiAgent_singleTurn_flightAgent-{domain}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )


@tool
def flight_search_assistant(query: str) -> str:
    """
    Process and respond to queries about flight searches, airport information, and travel routes.
    Use for ANY flight-related queries including direct flights, connecting flights, and airport details.
    
    Args:
        query: A question requiring flight searches or airport information
        
    Returns:
        Detailed flight options or airport information
    """
    try:
        # Create the specialized Flight Agent using the configuration you've provided
        flight_agent = flight_react_agent(flight_agent_tools)
        
        # Call the agent and return its response
        response = flight_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in flight_search_assistant: {str(e)}"

## reservation agent

In [8]:
reservation_tool_list = [
    'book_reservation',
    'cancel_reservation', 
    'get_reservation_details',
    'update_reservation_baggages',
    'update_reservation_flights',
    'update_reservation_passengers',
    'think'
]
reservation_agent_tools = [domain_tools[key] for key in reservation_tool_list]


def reservation_agent_prompt():
    
    system_prompt_template = """
You are the Reservation Agent for a travel website, specializing in managing the complete lifecycle of travel bookings from creation through modification to cancellation. 
Your expertise ensures seamless reservation management and transaction integrity throughout the booking process.
Use the provided tools to update, cancel, book, and get reservation details.

<capabilities>
- You can create new reservations through the book_reservation tool
- You can cancel existing reservations using the cancel_reservation tool
- You can retrieve comprehensive booking information via the get_reservation_details tool
- You can modify baggage allocations with the update_reservation_baggages tool
- You can change flight selections using the update_reservation_flights tool
- You can update passenger information through the update_reservation_passengers tool
- You can use the think tool for reasoning
</capabilities>

<instructions>
- You should not use made-up or placeholder arguments.
</instructions>

<policy>
{policy}
</policy>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt


def reservation_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def reservation_agent_tracing(domain):

    trace_attributes = {
        "langfuse.tags": [
            f"awsStrands-multiAgent_singleTurn_reservationAgent-{domain}",
        ]
    }

    return trace_attributes


def reservation_react_agent(tools):

    prompt = reservation_agent_prompt()
    model = reservation_agent_model()
    trace_attributes = reservation_agent_tracing(domain)

    return Agent(
        name = f"awsStrands-multiAgent_singleTurn_reservationAgent-{domain}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )


@tool
def reservation_management_assistant(query: str) -> str:
    """
    Process and respond to queries about booking, modifying, and canceling flight reservations.
    Use for ANY reservation-related queries including creating new bookings, updating passenger details, 
    changing flights, modifying baggage allowances, retrieving reservation information, and canceling bookings.
    
    Args:
        query: A question requiring reservation creation, modification, retrieval, or cancellation
        
    Returns:
        Confirmation of reservation actions or detailed booking information
    """
    try:
        # Create the specialized Reservation Agent using the configuration you've provided
        reservation_agent = reservation_react_agent(reservation_agent_tools)
        
        # Call the agent and return its response
        response = reservation_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in reservation_management_assistant: {str(e)}"

## supervisor agent

In [9]:
supervisor_tool_list = ['calculate', 'think', 'transfer_to_human_agents']
supervisor_tools = [domain_tools[key] for key in supervisor_tool_list]
supervisor_tools += [
    user_information_manager,
    flight_search_assistant,
    reservation_management_assistant,
]


def supervisor_agent_prompt():
    
    system_prompt_template = """
You are a helpful assistant for a travel website. Help the user answer any questions.

<capabilities>
- You have access to specialized agent teams through these tools:
  * user_information_manager: For queries about user profiles including date-of-birth, customer details, and certificates. You need to always pass user_id
  * flight_search_assistant: For flight searches, airport information, and travel routes
  * reservation_management_assistant: For booking, modifying, and canceling reservations
- You can perform calculations using the calculate tool
- You can use the think tool for complex reasoning and task breakdown
- You can escalate to human agents when necessary using transfer_to_human_agents
- You can ask for the user_id from the user
</capabilities>

<workflow_guidelines>
1. ANALYZE user requests to determine which specialized agent(s) should handle different aspects
2. DECOMPOSE complex multi-part requests into discrete subtasks
3. DELEGATE each subtask to the appropriate specialized agent
4. SYNTHESIZE information from multiple agents into coherent responses
5. ESCALATE to human agents when requests exceed automated capabilities
</workflow_guidelines>

<delegation_rules>
- For user account information, loyalty status, or certificate requests → user_information_manager
- For flight availability, routes, connections, or airport information → flight_search_assistant
- For creating, modifying, retrieving or canceling bookings → reservation_management_assistant
- For mathematical operations or price comparisons → calculate tool
- For complex reasoning or planning the approach → think tool
- For issues requiring human judgment or outside system capabilities → transfer_to_human_agents
</delegation_rules>

<instructions>
- Remeber to check if the the airport city is in the state mentioned by the user. For example, Houston is in Texas.
- Infer about the the U.S. state in which the airport city resides. For example, Houston is in Texas.
- You should not use made-up or placeholder arguments.
</instructions>

<policy>
{policy}
</policy>
"""

    prompt = system_prompt_template.format(policy = WIKI)

    return prompt


def supervisor_agent_model():

    model_id = "anthropic.claude-3-sonnet-20240229-v1:0" # "anthropic.claude-3-sonnet-20240229-v1:0" "anthropic.claude-3-5-sonnet-20240620-v1:0", "us.anthropic.claude-3-5-sonnet-20241022-v2:0" 

    return BedrockModel(
        model_id = model_id,
        region_name = region_name,
        max_tokens= 1024,
        temperature = 0.0,
        top_p = 1,
        boto_client_config=my_config,
    )


def supervisor_agent_tracing(user_id, session_id, domain):

    trace_attributes = {
        "user.id": user_id, 
        "session.id": session_id,
        "langfuse.tags": [
            user_id,
            session_id,
            f"awsStrands-multiAgent_singleTurn_supervisorAgent-{domain}",
        ]
    }

    return trace_attributes


def supervisor_react_agent(tools, user_id, session_id, domain):

    name = f"awsStrands-multiAgent_singleTurn_supervisorAgent-{domain}-{user_id}-{session_id}",
    prompt = supervisor_agent_prompt()
    model = supervisor_agent_model()
    trace_attributes = supervisor_agent_tracing(user_id, session_id, domain)

    return Agent(
        name = f"awsStrands-multiAgent_singleTurn_supervisorAgent-{domain}-{user_id}-{session_id}",
        model = model, 
        tools = tools, 
        system_prompt = prompt,
        trace_attributes = trace_attributes
    )

# Run

In [10]:
output_path = os.path.join("..", "data", "tau-bench", "tau_bench", "envs", f"{domain}", "tasks_singleturn.json")
with open(output_path, "r") as file:
    tasks = json.load(file)


In [11]:
# for index,task in enumerate(tasks):

index = 19
task = tasks[index]

index_str = str(index)
num_hashes = (50 - len(index_str) - 9) // 2
print(f"\n{'#' * num_hashes} Index:{index} {'#' * num_hashes}\n")

instruction = task['instruction']
print(f"Processing instruction: {instruction}")

user_id = task['user_id']
session_id= uuid.uuid4()
print(f"User ID: {user_id}\tSession ID: {session_id}\tDomain:{domain}")

user = simulated_user(instruction, user_id, session_id, domain)
agent = supervisor_react_agent(supervisor_tools, user_id, session_id, domain)

messages = run_user_agent(user, agent)
print(messages)

    # break


################### Index:19 ###################

Processing instruction: Your user id is raj_brown_5782 and you want to change your upcoming roundtrip flights which are currently DTW to LGA and back (reservation ID is VA5SGQ). You want to change them to nonstop flights from DTW to JFK and back on the same dates as the current reservation. Since you took insurance for this trip, you want change fees waived. You also want to add 1 checked bag. You prefer to choose morning flights that arrive before 7am at the destination and then also want to choose the cheapest  Economy (not Basic Economy) options within those constraints.
User ID: raj_brown_5782	Session ID: 09ade32b-32ee-468b-945d-1722bd81112f	Domain:airline


******** Agent ********

Hello! Welcome to the travel website. How can I assist you today with any flight bookings, reservations, or travel-related queries?{'user.id': 'raj_brown_5782'}


******** User *********

User Response: Hi, I need to change my upcoming roundtrip flights

In [40]:
agent.trace_span

_Span(name="invoke_agent awsStrands-multiAgent_singleTurn_supervisorAgent-airline-raj_brown_5782-09ade32b-32ee-468b-945d-1722bd81112f", context=SpanContext(trace_id=0x8d8f3b546d20170365700917f0dfbb0a, span_id=0xffae4590b7d079a9, trace_flags=0x01, trace_state=[], is_remote=False))